In [1]:
import cv2
import os
import pickle
from insightface.app import FaceAnalysis
from scipy.spatial.distance import cosine
import numpy as np
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import warnings
# 抑制检测albumentations的版本警告信息
warnings.filterwarnings("ignore", category=UserWarning, module="albumentations")

# Initialize InsightFace application
app = FaceAnalysis(providers=['CPUExecutionProvider'])  # 仅使用 CPU
app.prepare(ctx_id=0, det_size=(320, 320))  # Lower resolution for faster processing

# Path to load known face encodings and names
encodings_file_path = "./face_features/known_face_encodings.pkl"
names_file_path = "./face_features/known_face_names.pkl"

# Load known face encodings and names from file if they exist
if os.path.exists(encodings_file_path) and os.path.exists(names_file_path):
    with open(encodings_file_path, 'rb') as f:
        known_face_encodings = pickle.load(f)
    with open(names_file_path, 'rb') as f:
        known_face_names = pickle.load(f)
else:
    raise FileNotFoundError("The files known_face_encodings.pkl and known_face_names.pkl do not exist in ./face_features directory.")

# Initialize the camera
video_capture = cv2.VideoCapture(0)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
video_capture.set(cv2.CAP_PROP_FPS, 15)  # Set frame rate to 15 fps

frame_lock = threading.Lock()
latest_frame = None
detected_faces = []

def capture_frames():
    global latest_frame
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        with frame_lock:
            latest_frame = frame.copy()

capture_thread = threading.Thread(target=capture_frames)
capture_thread.daemon = True
capture_thread.start()

def match_faces(frame):
    global detected_faces
    faces = app.get(frame)
    matched_faces = []
    for face in faces:
        embedding = face.embedding
        name = "Unknown"

        # Calculate similarity with known faces
        similarities = [1 - cosine(embedding, known_face) for known_face in known_face_encodings]
        if similarities:
            best_match_index = similarities.index(max(similarities))
            if similarities[best_match_index] > 0.6:  # Set a similarity threshold
                name = known_face_names[best_match_index]

        # Store the face information for drawing later
        matched_faces.append({
            'bbox': face.bbox.astype(int),
            'name': name
        })
    detected_faces = matched_faces

# 使用 ipywidgets 显示视频流
image_widget = widgets.Image(format='jpeg')
display(image_widget)

try:
    while True:
        with frame_lock:
            frame = latest_frame.copy() if latest_frame is not None else None

        if frame is None:
            continue

        # Match faces in a separate thread
        matching_thread = threading.Thread(target=match_faces, args=(frame,))
        matching_thread.start()
        matching_thread.join()

        # Draw detected faces on the frame
        for face_info in detected_faces:
            box = face_info['bbox']
            name = face_info['name']

            # Draw a rectangle around the face
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)

            # Draw a label
            cv2.rectangle(frame, (box[0], box[3] - 35), (box[2], box[3]), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (box[0] + 6, box[3] - 6), font, 1.0, (255, 255, 255), 1)

        # 将帧转换为 JPEG 格式并更新 widget
        _, jpeg_frame = cv2.imencode('.jpg', frame)
        image_widget.value = jpeg_frame.tobytes()

        # 添加延迟，减少刷新频率
        time.sleep(0.05)

except KeyboardInterrupt:
    print("Stopping...")

# Release the camera and close all windows
video_capture.release()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/asus/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/asus/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/asus/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/asus/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/asus/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size

Image(value=b'', format='jpeg')

Stopping...
